In [86]:
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import requests 
from collections import defaultdict
from ipynb.fs.full.transportationNetworks import bounding_coordinates, flatten_coords
import csv

In [98]:
def count_objects_in_polygon(polygon):
    if not isinstance(polygon, (Polygon, MultiPolygon)):
        raise TypeError("Not polygon obj")
    
    shop_types = ['grocery', 'general', 'supermarket', 'deli', 'greengrocer', 'food', 'marketplace', 'kiosk']
    try:
        grocery_stores = ox.features.features_from_polygon(polygon, tags={"shop": shop_types, "amenity": shop_types})
    except ox._errors.InsufficientResponseError:
        return {a:0 for a in shop_types}
    shop_counts = defaultdict(int)
    
    if not grocery_stores.empty:
        for _, store in grocery_stores.iterrows():
            shop_type = store.get('shop') or store.get('amenity')
            if shop_type in shop_types:
                shop_counts[shop_type] += 1
    
    for shop_type in shop_types:
        if shop_type not in shop_counts:
            shop_counts[shop_type] = 0
    
    return dict(shop_counts)


In [99]:
def return_weight(city, country):
    poly_coords = bounding_coordinates(city, country)[0]
    poly_coords = flatten_coords(poly_coords)
    poly = Polygon(poly_coords)


    object_counts = count_objects_in_polygon(poly)
    square_surface_area = poly.area
    
    weightSum = object_counts['grocery']*3 + object_counts['general']*3 + object_counts['supermarket']*3 + object_counts['deli'] *2 + object_counts['greengrocer'] * 2 + object_counts['food'] * 1 + object_counts['marketplace']*3 + object_counts['kiosk']*1

    return weightSum/square_surface_area


In [100]:
def test():
    print(return_weight('Cidade De Chimoio', 'Mozambique'))


In [101]:

def update_grocery_density(csv_filepath):
    # Read the CSV data
    with open(csv_filepath, newline='') as file:
        reader = csv.DictReader(file)
        rows = list(reader)  # Convert iterator to list to reuse it

    for row in rows:
        print("Checking:", row['Region'].replace("_", " "), row['Country'].replace("_", " "))
        result = return_weight(row['Region'].replace("_", " "), row['Country'].replace("_", " "))
        print(row['Region'], row['Country'], result)
        row['Food_Storage_Groceries'] = float(row['Food_Storage_Groceries']) + result

    with open(csv_filepath, 'w', newline='') as file:
        fieldnames = rows[0].keys()  # Grab the field names from the first row
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

In [102]:
update_grocery_density("/Users/alexforman/Downloads/foodwatchInput.csv")

Checking: Bamingui Central African Republic


/opt/homebrew/lib/python3.12/site-packages/osmnx/_overpass.py:245: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


3 4.737893839909495
Bamingui Central_African_Republic 0.6331927437313173
Checking: Bangui Central African Republic
12 0.005635329566469988
Bangui Central_African_Republic 2129.4229305415565
Checking: Badakhshan Afghanistan


/opt/homebrew/lib/python3.12/site-packages/osmnx/_overpass.py:245: UserWarning: This area is 34 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


111 4.399299311068158
Badakhshan Afghanistan 25.231290746855095
Checking: Badghis Afghanistan
0 2.047361402595349
Badghis Afghanistan 0.0
Checking: Baghlan Afghanistan
0 1.7804168209671485
Baghlan Afghanistan 0.0
Checking: Balkh Afghanistan
0 0.05171428141380005
Balkh Afghanistan 0.0
Checking: Bamyan Afghanistan
2 1.7583558770541585
Bamyan Afghanistan 1.137426175269296
